In [ ]:
from warnings import simplefilter
# ignore all future warnings
simplefilter(action='ignore', category=UserWarning)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pickle
import pandas as pd
import numpy as np
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
from tqdm.notebook import tqdm
nltk.download('vader_lexicon')
!pip install transformers
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from scipy.special import softmax

sia = SentimentIntensityAnalyzer()

# This Hugging Face Transformers library to load a pre-trained sentiment analysis model for Twitter data.
task='sentiment'
MODEL = f"cardiffnlp/twitter-roberta-base-{task}"
tokenizer = AutoTokenizer.from_pretrained(MODEL) # Convert the raw text into a format so that the model can process
model = AutoModelForSequenceClassification.from_pretrained(MODEL)

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 30.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 68.9 MB/s eta 0:00:00


In [ ]:
with open('/content/drive/MyDrive/Sentiment Analysis using Tweets-20230225T215534Z-001/Sentiment Analysis using Tweets/TwitterSentimentModel', 'rb') as f:
  myModel = pickle.load(f)

In [ ]:
knn = myModel['knn']
lr = myModel['lr'] # Logistic Regression
rf = myModel['rf']
mlr = myModel['mlr'] # Multiple Linear Regrssion 

In [ ]:
def getAllPolarityScore(text):
  vadarScore = sia.polarity_scores(text)
  encoded_text = tokenizer(text,return_tensors='pt')
  output = model(**encoded_text)
  scores = output[0][0].detach().numpy()
  scores = softmax(scores)
  scores_dict = {
      "roberta_neg" : scores[0],
      "roberta_neu" : scores[1],
      "roberta_pos" : scores[2]
  }
  vadarList =  [i for i in vadarScore.values()]
  robertaList =[i for i in scores_dict.values()]
  knn_predict = knn.predict([(vadarList + robertaList)])
  linearReggPredict = mlr.predict([(vadarList + robertaList)])
  logisticReggPredict = lr.predict([(vadarList + robertaList)])
  randomForestPredict = rf.predict([(vadarList + robertaList)])

  return {'TAG' : { "KNN":knn_predict,"RandomForest":randomForestPredict,"Logistic Regression":logisticReggPredict }, 'SCORE' :  linearReggPredict }


In [ ]:
getAllPolarityScore("I am happy")

{'TAG': {'KNN': array([1]),
  'RandomForest': array([1]),
  'Logistic Regression': array([1])},
 'SCORE': array([0.98161929])}

In [ ]:
getAllPolarityScore("I am Sad")

{'TAG': {'KNN': array([0]),
  'RandomForest': array([0]),
  'Logistic Regression': array([0])},
 'SCORE': array([0.9937464])}

In [ ]:
getAllPolarityScore("G20 finance meet ends, no consensus on Ukraine")

{'TAG': {'KNN': array([0]),
  'RandomForest': array([0]),
  'Logistic Regression': array([0])},
 'SCORE': array([0.94295544])}

In [ ]:
getAllPolarityScore("‘India ready to contribute to peace process’: PM Modi on Ukraine crisis")

{'TAG': {'KNN': array([1]),
  'RandomForest': array([1]),
  'Logistic Regression': array([1])},
 'SCORE': array([0.91553413])}